# Notebook

## Init

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import optim

## Cuda

In [ ]:
torch.__version__, torch.cuda.is_available(), torch.version.cuda

In [ ]:
cuda_id = torch.cuda.current_device()

In [ ]:
torch.cuda.current_device(), torch.cuda.get_device_name(cuda_id)

## Dataset

In [ ]:
batch_size_train = 60
batch_size_test = 1000
learning_rate = 0.01
NUM_WORKERS=1
momentum = 0.9
ngpu = 1 
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
train_data = torchvision.datasets.MNIST('./data/', train=True, download=True,
                             transform=torchvision.transforms.ToTensor())

test_data = torchvision.datasets.MNIST('./data/', train=False, download=True,
                             transform=torchvision.transforms.ToTensor())

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size_train, shuffle=True, num_workers=NUM_WORKERS)

test_loader = DataLoader(test_data, batch_size=batch_size_test, shuffle=True, num_workers=NUM_WORKERS)

In [ ]:
plt.imshow(train_data.data[0]) #use cmap='gray' to view in grayscale
plt.title('%i' % train_data.targets[0])
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
ex = enumerate(train_loader)
batch_idx, (ex_data, ex_targets) = next(ex)

In [ ]:
ex_data.shape

In [ ]:
plt.imshow(ex_data[0][0], cmap='gray', interpolation='none')
plt.title('%i' % ex_targets[0])
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=2
            ),
            nn.ReLU(),
            nn.Dropout2d(),                      
            nn.MaxPool2d(2),
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=10, out_channels=32, kernel_size=5, stride=1, padding=2
            ),
            nn.ReLU(),
            nn.Dropout2d(),                      
            nn.MaxPool2d(2),
        )

        self.fc1 = nn.Sequential(
            nn.Linear(1568, 50),
                nn.ReLU(),
                nn.Dropout(),
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(50, 10),
            nn.LogSoftmax(),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        output = self.fc2(x)
        return output

In [ ]:
Net = Net()

In [ ]:
loss_crit = nn.CrossEntropyLoss()   
loss_crit

In [ ]:
optimizer = optim.SGD(Net.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
#Net.to(device)

## Training

In [ ]:
def train(epochs, net):
    
    net.train()

    total = len(train_loader)

    for epoch in range(epochs):

        for i, (data, label) in enumerate(train_loader):
            optimizer.zero_grad()
            out = net(data)
            loss = loss_crit(out, label)
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, epochs, i + 1, total, loss.item()))
                torch.save(net.state_dict(), './results/model.pth')
                torch.save(optimizer.state_dict(), './results/optimizer.pth')

In [ ]:
num_epoch = 25

train(num_epoch, Net)

## Testing

In [ ]:
def test(net):
    net.eval()
    correct = 0 
    with torch.no_grad():
        for data, target in test_loader:
            out = net(data)
            pred = torch.max(out, 1)[1].data.squeeze()
            correct = correct + pred.eq(target.data.view_as(pred)).sum()
            accuracy = (pred == target).sum().item() / float(target.size(0))
    print("Accuracy = %.3f" % accuracy) 
    print("Correct = %.f" % correct) 

In [ ]:
test(Net)

In [ ]:
test_img = next(iter(test_loader))
test_data, test_label = test_img

In [ ]:
with torch.no_grad():
    actual_number = test_label[:5]
    test_output = Net(test_data[:5])
    pred = torch.max(test_output, 1)[1]
pred

In [ ]:
fig = plt.figure()
for i in range(5):
    plt.subplot(2,3,i+1)
    plt.tight_layout()
    plt.imshow(test_data[:5][i][0], cmap='gray', interpolation='none')
    plt.title("Prediction: {}".format(pred[i]))
    plt.xticks([])
    plt.yticks([])

print(f'Predictions: {pred}')
print(f'Actual: {actual_number}')